<a href="https://colab.research.google.com/github/PHRCpvh/BERT-Perguntas-e-Respostas/blob/master/BERT_Perguntas_e_Respostas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 - Instalação das Bibliotecas

• SQUAD Link: https://rajpurkar.github.io/SQuAD-explorer/

In [ ]:
!pip install sentencepiece
!pip install tf-models-nightly
!pip install tf-nightly

     |████████████████████████████████| 1.1MB 3.3MB/s 
     |████████████████████████████████| 1.0MB 3.4MB/s 
     |████████████████████████████████| 36.6MB 113kB/s 
     |████████████████████████████████| 102kB 11.6MB/s 
     |████████████████████████████████| 276kB 44.1MB/s 
     |████████████████████████████████| 358kB 44.0MB/s 
     |████████████████████████████████| 390.1MB 40kB/s 
     |████████████████████████████████| 174kB 48.0MB/s 
     |████████████████████████████████| 10.2MB 42.3MB/s 
     |████████████████████████████████| 460kB 45.8MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-7.0.0-cp36-none-any.whl size=20071 sha256=80b61de908a1dfe587fa7fedb9f0487295a3a385eb5d616157e8e91b4045f882
  Stored in directory: /root/.cache/pip/wheels/f1/93/7b/127daf0c3a5a49feb2fecd468d508067c733fba5192f726ad1
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=df7c9250daf602d8d62bb907c281338b8fda5b510740e128d5cfd62b56c26163
  Stored in di

In [ ]:
!#pip install tensorflow==2.2.0

In [ ]:
import tensorflow as tf
tf.__version__

'2.4.0-dev20200916'

In [ ]:
import tensorflow_hub as hub
from official.nlp.bert.tokenization import FullTokenizer
from official.nlp.bert.input_pipeline import create_squad_dataset
from official.nlp.data.squad_lib import generate_tf_record_from_json_file
from official.nlp import optimization
from official.nlp.data.squad_lib import read_squad_examples
from official.nlp.data.squad_lib import FeatureWriter
from official.nlp.data.squad_lib import convert_examples_to_features
from official.nlp.data.squad_lib import write_predictions

In [ ]:
import numpy as np
import math
import random
import time
import json
import collections
import os
from google.colab import drive

# 2 - Pré-processamento

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
input_meta_data = generate_tf_record_from_json_file(
    "/content/drive/My Drive/Google Colab/BERT/train-v1.1.json",
    "/content/drive/My Drive/Google Colab/BERT/vocab.txt",
    "/content/drive/My Drive/Google Colab/BERT/train-v1.1.tf_record"
)

In [ ]:
with tf.io.gfile.GFile("/content/drive/My Drive/Google Colab/BERT/train_meta_data", "w") as writer:
  writer.write(json.dumps(input_meta_data, indent=4) + "\n")

In [ ]:
BATCH_SIZE = 4
train_dataset = create_squad_dataset(
    "/content/drive/My Drive/Google Colab/BERT/train-v1.1.tf_record",
    input_meta_data["max_seq_length"], # 384
    BATCH_SIZE,
    is_training = True
)

# 3 - Construção do Modelo

Camada SQUAD

In [ ]:
 class BertSquadLayer(tf.keras.layers.Layer):
  def __init__(self):
    super(BertSquadLayer, self).__init__()
    self.final_dense = tf.keras.layers.Dense(units=2,
                                             kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))
    
  def call(self, inputs):
    logits = self.final_dense(inputs) # (batch_size, seq_len, 2)
    logits = tf.transpose(logits, [2, 0, 1]) # (2, batch_size, seq_len)
    unstacked_logits = tf.unstack(logits, axis=0) # [(batch_size, seq_len), (batch_size, seq_len)]
    return unstacked_logits[0], unstacked_logits[1]

Modelo Completo

In [ ]:
class BERTSquad(tf.keras.Model):
  def __init__(self, name="bert_squad"):
    super(BERTSquad, self).__init__(name=name)
    self.bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                                     trainable = True)
    self.squad_layer = BertSquadLayer()

  def apply_bert(self, inputs):
    _, sequence_output = self.bert_layer([inputs["input_word_ids"],
                                          inputs["input_mask"],
                                          inputs["input_type_ids"]])
    return sequence_output

  def call(self, inputs):
    seq_outputs = self.apply_bert(inputs)
    start_logits, end_logits = self.squad_layer(seq_outputs)
    return start_logits, end_logits

# 4 - Treinamento

Criação da IA

In [ ]:
TRAIN_DATA_SIZE = 88641
NB_BATCHES_TRAIN = 2000
BATCH_SIZE = 4
NB_EPOCHS = 3
INIT_LR = 5e-5
WARMUP_STEPS = int(NB_BATCHES_TRAIN * 0.1)

In [ ]:
train_dataset_light = train_dataset.take(NB_BATCHES_TRAIN)

In [ ]:
bert_squad = BERTSquad()

In [ ]:
optimizer = optimization.create_optimizer(init_lr=INIT_LR,
                                          num_train_steps = NB_BATCHES_TRAIN,
                                          num_warmup_steps = WARMUP_STEPS)

In [ ]:
def squad_loss_fn(labels, model_outputs):
  start_positions = labels['start_positions']
  end_positions = labels['end_positions']
  start_logits, end_logits = model_outputs

  start_loss = tf.keras.backend.sparse_categorical_crossentropy(start_positions, start_logits, from_logits=True)
  end_loss = tf.keras.backend.sparse_categorical_crossentropy(end_positions, end_logits, from_logits=True)

  total_loss = (tf.reduce_mean(start_loss) + tf.reduce_mean(end_loss)) / 2

  return total_loss

In [ ]:
train_loss = tf.keras.metrics.Mean(name="train_loss")

In [ ]:
bert_squad.compile(optimizer, squad_loss_fn)

In [ ]:
checkpoint_path = "/content/drive/My Drive/Google Colab/BERT/Q&A"
ckpt = tf.train.Checkpoint(bert_squad = bert_squad)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep = 1)
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest checkpoint restored!")

Latest checkpoint restored!


Treinamento Personalizado

In [ ]:
for epoch in range(NB_EPOCHS):
  print("Start of epoch {}".format(epoch + 1))
  start = time.time()

  train_loss.reset_states()

  for (batch, (inputs, targets)) in enumerate(train_dataset_light):
    with tf.GradientTape() as tape:
      model_outputs = bert_squad(inputs)
      loss = squad_loss_fn(targets, model_outputs)

    gradients = tape.gradient(loss, bert_squad.trainable_variables)
    optimizer.apply_gradients(zip(gradients, bert_squad.trainable_variables))

    train_loss(loss)

    if batch % 50 == 0:
      print("Epoch {} Batch {} Loss {} (:.4f)".format(epoch+1, batch, train_loss.result()))

    if batch % 500 == 0:
      ckpt_save_path = ckpt_manager.save()
      print("Saving checkpoint for epoch {} at {}".format(epoch+1, ckpt_save_path))

  print("Time taken for 1 epoch: {} secs\n".format(time.time() - start))

Start of epoch 1
Epoch 1 Batch 0 Loss 6.132259368896484 (:.4f)
Saving checkpoint for epoch 1 at /content/drive/My Drive/Google Colab/BERT/Q&A/ckpt-9
Epoch 1 Batch 50 Loss 5.853891372680664 (:.4f)
Epoch 1 Batch 100 Loss 5.3107500076293945 (:.4f)
Epoch 1 Batch 150 Loss 4.639137268066406 (:.4f)
Epoch 1 Batch 200 Loss 4.116105079650879 (:.4f)
Epoch 1 Batch 250 Loss 3.695465564727783 (:.4f)
Epoch 1 Batch 300 Loss 3.4021105766296387 (:.4f)
Epoch 1 Batch 350 Loss 3.247800350189209 (:.4f)
Epoch 1 Batch 400 Loss 3.0461175441741943 (:.4f)
Epoch 1 Batch 450 Loss 2.88688325881958 (:.4f)
Epoch 1 Batch 500 Loss 2.743670701980591 (:.4f)
Saving checkpoint for epoch 1 at /content/drive/My Drive/Google Colab/BERT/Q&A/ckpt-10
Epoch 1 Batch 550 Loss 2.629446268081665 (:.4f)
Epoch 1 Batch 600 Loss 2.542201042175293 (:.4f)
Epoch 1 Batch 650 Loss 2.4675955772399902 (:.4f)
Epoch 1 Batch 700 Loss 2.3980979919433594 (:.4f)
Epoch 1 Batch 750 Loss 2.328103542327881 (:.4f)
Epoch 1 Batch 800 Loss 2.274634838104248 

KeyboardInterrupt: ignored

In [ ]:
to_restore = tf.Variable(tf.zeros([5]))
print(to_restore.numpy())  # All zeros
fake_layer = tf.train.Checkpoint(bias=to_restore)
fake_net = tf.train.Checkpoint(l1=fake_layer)
new_root = tf.train.Checkpoint(net=fake_net)
status = new_root.restore(tf.train.latest_checkpoint("/content/drive/My Drive/Google Colab/BERT/Q&A/ckpt-12.data-00000-of-00001"))
print(to_restore.numpy())  # We get the restored value now

[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]


# 5 - Avaliação

Preparação

In [ ]:
eval_examples = read_squad_examples(
    "/content/drive/My Drive/Google Colab/BERT/dev-v1.1.json",
    is_training = False,
    version_2_with_negative = False)



> Definição da função para gerar o tf_record



In [ ]:
eval_writer = FeatureWriter(
    filename=os.path.join("/content/drive/My Drive/Google Colab/BERT",
                          "eval.tf_record"),
    is_training = False)



> Tokenizador



In [ ]:
my_bert_layer = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
    trainable = False)
vocab_file = my_bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = my_bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)



> Caracterizador da lista "eva_features"



In [ ]:
def _append_feature(feature, is_padding):
  if not is_padding:
      eval_features.append(feature)
  eval_writer.process_feature(feature)



> Caracterizador do arquivo tf.record



In [ ]:
eval_features = []
dataset_size = convert_examples_to_features(
    examples = eval_examples,
    tokenizer = tokenizer,
    max_seq_length = 384,
    doc_stride = 128,
    max_query_length = 64,
    is_training = False,
    output_fn = _append_feature,
    batch_size = 4)

In [ ]:
eval_writer.close()



> Base de dados



In [ ]:
BATCH_SIZE = 4

eval_dataset = create_squad_dataset(
    "/content/drive/My Drive/Google Colab/BERT/eval.tf_record",
    384, # input_meta_data['max_seq_length'],
    BATCH_SIZE,
    is_training = False)

Previsões com funções do Google

In [ ]:
RawResult = collections.namedtuple("RawResult",
                                   ["unique_id", "start_logits", "end_logits"])

In [ ]:
def get_raw_results(predictions):
  for unique_ids, start_logits, end_logits in zip(predictions["unique_ids"],
                                                  predictions["start_logits"],
                                                  predictions["end_logits"]):
    yield RawResult(
        unique_id=unique_ids.numpy(),
        start_logits=start_logits.numpy().tolist(),
        end_logits=end_logits.numpy().tolist()
    )

In [ ]:
all_results = []
for count, inputs in enumerate(eval_dataset):
  x, _ = inputs
  #print(x)
  unique_ids = x.pop("unique_ids")
  start_logits, end_logits = bert_squad(x, training = False)
  output_dict = dict(
      unique_ids = unique_ids,
      start_logits = start_logits,
      end_logits = end_logits
  )
  for result in get_raw_results(output_dict):
    all_results.append(result)

  print(count)
  if count % 100 == 0:
    print("{}/{}".format(count, 2709))

0
0/2709
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
100/2709
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
200/2709
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
2

In [ ]:
len(all_results) #Usar após o previsor contabilizar todos os resultados da fórmula anterior.

10836

In [ ]:
all_results[0]

RawResult(unique_id=1000000000, start_logits=[-5.680498123168945, -6.508359909057617, -6.347847938537598, -6.409255027770996, -6.601614475250244, -6.679850101470947, -6.885415554046631, -6.44765043258667, -5.096132278442383, -6.475470066070557, -6.751291275024414, -6.835025310516357, -6.549988746643066, -2.6234476566314697, -5.800776481628418, -5.870785713195801, -6.058463096618652, -5.638486385345459, -4.699994087219238, -5.976578712463379, -6.298312187194824, -5.913740158081055, -5.703412055969238, -5.316441059112549, -4.532556533813477, -6.576051712036133, -5.578375816345215, -3.922252893447876, -6.0888142585754395, -6.634547233581543, -6.550261974334717, -5.494431018829346, -6.453731536865234, -6.363158226013184, -5.866878032684326, -5.3049139976501465, -6.713850975036621, -6.036337852478027, -1.3589874505996704, 2.1116151809692383, -5.711420059204102, -6.098797798156738, -6.323404788970947, -3.880269765853882, -4.868359565734863, -3.1288504600524902, 4.335103988647461, -1.77936625

In [ ]:
output_prediction_file = "/content/drive/My Drive/Google Colab/BERT/predictions.json"
output_nbest_file = "/content/drive/My Drive/Google Colab/BERT/nbest_predictions.json"
output_null_log_odds_file = "/content/drive/My Drive/Google Colab/BERT/null_odds.json"

In [ ]:
write_predictions(
    eval_examples,
    eval_features,
    all_results,
    20,
    30,
    True,
    output_prediction_file,
    output_nbest_file,
    output_null_log_odds_file,
    verbose=False)

Fazer previsões personalizadas



> Dicionário de Inputs - Essencial


In [ ]:
my_bert_layer = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
    trainable = False)
vocab_file = my_bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = my_bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [ ]:
def is_whitespace(c):
  if c == " " or c == "\t" or c == "\r" or c == "\n" or ord(c) == 0x202F:
    return True
  return False

In [ ]:
is_whitespace("a")

False

In [ ]:
is_whitespace("\n")

True

In [ ]:
is_whitespace(" ")

True

In [ ]:
def whitespace_split(text):
  doc_tokens = []
  prev_is_whitespace = True
  for c in text:
    if is_whitespace(c):
      prev_is_whitespace = True
    else:
      if prev_is_whitespace:
        doc_tokens.append(c)
      else:
        doc_tokens[-1] += c
      prev_is_whitespace = False
  return doc_tokens

In [ ]:
whitespace_split("My dog likes strawberries.")

['My', 'dog', 'likes', 'strawberries.']

In [ ]:
tokenizer.tokenize("my")

['my']

In [ ]:
tokenizer.tokenize("strawberries")

['straw', '##berries']

In [ ]:
t = tokenizer.tokenize("strawberries")
t

['straw', '##berries']

In [ ]:
len(t)

2

In [ ]:
[1] * len(t)

[1, 1]

In [ ]:
def tokenize_context(text_words):
  text_tok = []
  tok_to_word_id = []
  for word_id, word in enumerate(text_words):
    word_tok = tokenizer.tokenize(word)
    text_tok += word_tok
    tok_to_word_id += [word_id] * len(word_tok)
  return text_tok, tok_to_word_id

In [ ]:
tokenize_context(whitespace_split("My dog likes strawberries."))

(['my', 'dog', 'likes', 'straw', '##berries', '.'], [0, 1, 2, 3, 3, 3])

In [ ]:
def get_ids(tokens):
    return tokenizer.convert_tokens_to_ids(tokens)

def get_mask(tokens):
    return np.char.not_equal(tokens, "[PAD]").astype(int)

def get_segments(tokens):
    seg_ids = []
    current_seg_id = 0
    for tok in tokens:
        seg_ids.append(current_seg_id)
        if tok == "[SEP]":
           current_seg_id = 1 - current_seg_id
    return seg_ids

In [ ]:
my_context = '''One of its earliest massive implementations was brought about by Egyptians against the British occupation in the 1919 Revolution. Civil disobedience is one of the many ways people have rebelled against what they deem to be unfair laws. It has been used in many nonviolent resistance movements in India (Gandhi's campaigns for independence from the British Empire), in Czechoslovakia's Velvet Revolution and in East Germany to oust their communist governments, In South Africa in the fight against apartheid, in the American Civil Rights Movement, in the Singing Revolution to bring independence to the Baltic countries from the Soviet Union, recently with the 2003 Rose Revolution in Georgia and the 2004 Orange Revolution in Ukraine, among other various movements worldwide.'''

One of its earliest massive implementations was brought about by Egyptians against the British occupation in the 1919 Revolution. Civil disobedience is one of the many ways people have rebelled against what they deem to be unfair laws. It has been used in many nonviolent resistance movements in India (Gandhi's campaigns for independence from the British Empire), in Czechoslovakia's Velvet Revolution and in East Germany to oust their communist governments, In South Africa in the fight against apartheid, in the American Civil Rights Movement, in the Singing Revolution to bring independence to the Baltic countries from the Soviet Union, recently with the 2003 Rose Revolution in Georgia and the 2004 Orange Revolution in Ukraine, among other various movements worldwide.

In [ ]:
my_question = '''What is civil disobedience?'''
#my_question = '''When the civil disobedience began?'''
#my_question = '''What was the the movement called that brought Baltic countries independence from the Soviet Union?'''

In [ ]:
def create_input_dict(question, context):
  question_tok = tokenizer.tokenize(my_question)
  #print(question_tok)

  context_words = whitespace_split(context)
  #print(context_words)
  context_tok, context_tok_to_word_id = tokenize_context(context_words)
  #print(context_tok)
  #print(context_tok_to_word_id)

  input_tok = question_tok + ["[SEP]"] + context_tok + ["[SEP]"]
  #print(input_tok)
  input_tok += ["[PAD]"]*(384-len(input_tok))
  #print(input_tok)

  input_dict = {}
  input_dict["input_word_ids"] = tf.expand_dims(tf.cast(get_ids(input_tok), tf.int32), 0)
  input_dict["input_mask"] = tf.expand_dims(tf.cast(get_mask(input_tok), tf.int32), 0)
  input_dict["input_type_ids"] = tf.expand_dims(tf.cast(get_segments(input_tok), tf.int32), 0)

  return input_dict, context_words, context_tok_to_word_id, len(question_tok)

In [ ]:
my_input_dict, my_context_words, context_tok_to_word_id, question_tok_len = create_input_dict(my_question, my_context)

In [ ]:
my_input_dict["input_word_ids"]

<tf.Tensor: shape=(1, 384), dtype=int32, numpy=
array([[ 2054,  2003,  2942,  4487,  6499,  8270, 13684,  1029,   102,
         2028,  1997,  2049,  5700,  5294, 24977,  2001,  2716,  2055,
         2011, 23437,  2114,  1996,  2329,  6139,  1999,  1996,  4529,
         4329,  1012,  2942,  4487,  6499,  8270, 13684,  2003,  2028,
         1997,  1996,  2116,  3971,  2111,  2031, 25183,  2114,  2054,
         2027,  9266,  2213,  2000,  2022, 15571,  4277,  1012,  2009,
         2038,  2042,  2109,  1999,  2116,  2512, 25500, 16136,  5012,
         5750,  1999,  2634,  1006, 12338,  1005,  1055,  8008,  2005,
         4336,  2013,  1996,  2329,  3400,  1007,  1010,  1999, 12833,
         1005,  1055, 10966,  4329,  1998,  1999,  2264,  2762,  2000,
        15068,  3367,  2037,  4750,  6867,  1010,  1999,  2148,  3088,
         1999,  1996,  2954,  2114, 17862,  1010,  1999,  1996,  2137,
         2942,  2916,  2929,  1010,  1999,  1996,  4823,  4329,  2000,
         3288,  4336,  2000, 

In [ ]:
my_input_dict["input_mask"]

<tf.Tensor: shape=(1, 384), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
my_input_dict["input_type_ids"]

<tf.Tensor: shape=(1, 384), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
print(my_context_words)

['One', 'of', 'its', 'earliest', 'massive', 'implementations', 'was', 'brought', 'about', 'by', 'Egyptians', 'against', 'the', 'British', 'occupation', 'in', 'the', '1919', 'Revolution.', 'Civil', 'disobedience', 'is', 'one', 'of', 'the', 'many', 'ways', 'people', 'have', 'rebelled', 'against', 'what', 'they', 'deem', 'to', 'be', 'unfair', 'laws.', 'It', 'has', 'been', 'used', 'in', 'many', 'nonviolent', 'resistance', 'movements', 'in', 'India', "(Gandhi's", 'campaigns', 'for', 'independence', 'from', 'the', 'British', 'Empire),', 'in', "Czechoslovakia's", 'Velvet', 'Revolution', 'and', 'in', 'East', 'Germany', 'to', 'oust', 'their', 'communist', 'governments,', 'In', 'South', 'Africa', 'in', 'the', 'fight', 'against', 'apartheid,', 'in', 'the', 'American', 'Civil', 'Rights', 'Movement,', 'in', 'the', 'Singing', 'Revolution', 'to', 'bring', 'independence', 'to', 'the', 'Baltic', 'countries', 'from', 'the', 'Soviet', 'Union,', 'recently', 'with', 'the', '2003', 'Rose', 'Revolution', 'in

In [ ]:
print(context_tok_to_word_id)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 18, 19, 20, 20, 20, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 33, 34, 35, 36, 37, 37, 38, 39, 40, 41, 42, 43, 44, 44, 44, 45, 46, 47, 48, 49, 49, 49, 49, 50, 51, 52, 53, 54, 55, 56, 56, 56, 57, 58, 58, 58, 59, 60, 61, 62, 63, 64, 65, 66, 66, 67, 68, 69, 69, 70, 71, 72, 73, 74, 75, 76, 77, 77, 78, 79, 80, 81, 82, 83, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 113, 114, 115, 116, 117, 118, 118]


In [ ]:
question_tok_len

8

Previsões

In [ ]:
#end_logits, start_logits   = bert_squad(my_input_dict, training = False)

In [ ]:
start_logits, end_logits = bert_squad(my_input_dict, training = False)

In [ ]:
start_logits[0]

<tf.Tensor: shape=(384,), dtype=float32, numpy=
array([-4.695386 , -5.34116  , -4.3556075, -6.304845 , -6.235936 ,
       -6.5309753, -6.9285088, -6.1335783, -6.4121313, -5.566921 ,
       -6.3553395, -5.4073553, -5.5902686, -5.92128  , -6.1695895,
       -6.2080164, -5.87314  , -6.3338823, -5.7698708, -5.108214 ,
       -5.8242555, -5.7879515, -5.3539906, -6.5561504, -6.011974 ,
       -5.33072  , -5.3127875, -6.6688704, -5.7903996, -3.595606 ,
       -5.1396422, -5.117301 , -5.3041697, -5.664213 , -2.7623408,
       -0.6368595, -4.210598 , -2.6133614, -4.037932 , -3.284277 ,
       -3.7254372, -5.0051594, -2.9122784, -4.7608585, -4.117655 ,
       -5.2258253, -5.353958 , -6.1201267, -5.4820824, -5.1737804,
       -4.5405073, -4.9879394, -5.870536 , -4.427004 , -5.983035 ,
       -6.13952  , -5.9632416, -5.6378336, -5.11422  , -4.96364  ,
       -6.3711033, -6.641406 , -5.9436207, -6.5833116, -6.132853 ,
       -5.629445 , -6.290721 , -5.2178154, -6.5651474, -6.6541653,
       -6.3235

In [ ]:
question_tok_len

8

In [ ]:
start_logits_context = start_logits.numpy()[0, question_tok_len + 1:]

In [ ]:
start_logits_context

array([-5.566921 , -6.3553395, -5.4073553, -5.5902686, -5.92128  ,
       -6.1695895, -6.2080164, -5.87314  , -6.3338823, -5.7698708,
       -5.108214 , -5.8242555, -5.7879515, -5.3539906, -6.5561504,
       -6.011974 , -5.33072  , -5.3127875, -6.6688704, -5.7903996,
       -3.595606 , -5.1396422, -5.117301 , -5.3041697, -5.664213 ,
       -2.7623408, -0.6368595, -4.210598 , -2.6133614, -4.037932 ,
       -3.284277 , -3.7254372, -5.0051594, -2.9122784, -4.7608585,
       -4.117655 , -5.2258253, -5.353958 , -6.1201267, -5.4820824,
       -5.1737804, -4.5405073, -4.9879394, -5.870536 , -4.427004 ,
       -5.983035 , -6.13952  , -5.9632416, -5.6378336, -5.11422  ,
       -4.96364  , -6.3711033, -6.641406 , -5.9436207, -6.5833116,
       -6.132853 , -5.629445 , -6.290721 , -5.2178154, -6.5651474,
       -6.6541653, -6.323512 , -6.42209  , -5.9281716, -6.889344 ,
       -6.586534 , -6.3654556, -7.3317366, -7.222248 , -6.54464  ,
       -5.240746 , -5.236367 , -6.2708178, -6.382648 , -5.6517

In [ ]:
end_logits_context = end_logits.numpy()[0, question_tok_len + 1:]

In [ ]:
end_logits_context

array([-6.7446074, -6.515649 , -6.458913 , -6.4504395, -6.3032613,
       -5.0494056, -6.626792 , -6.5562367, -5.7418675, -6.692408 ,
       -5.0974126, -6.119023 , -6.1501546, -5.0221753, -4.0058556,
       -6.2092185, -6.435049 , -5.6676536, -3.2516017, -2.7163634,
       -5.619785 , -5.8037176, -5.734294 , -5.5066605, -3.580192 ,
       -6.1690755, -5.683391 , -6.221909 , -6.2033877, -5.412827 ,
       -5.154177 , -4.993631 , -5.465799 , -3.9264193, -4.735207 ,
       -5.657402 , -6.197018 , -5.514172 , -4.8591385, -6.355208 ,
       -5.651754 , -4.0220866,  1.0050524, -3.05205  , -6.3515778,
       -6.5526915, -6.441319 , -6.0244117, -6.791863 , -6.8768573,
       -6.8974123, -6.3614044, -5.666736 , -6.0473213, -5.209497 ,
       -6.550528 , -4.7900233, -6.344604 , -6.3565087, -6.376479 ,
       -6.1835933, -6.237684 , -6.451015 , -6.1518364, -6.1440225,
       -6.3175135, -6.254434 , -4.18049  , -4.5349164, -6.1386423,
       -6.920312 , -6.169705 , -6.426493 , -6.183582 , -6.4993

In [ ]:
print(context_tok_to_word_id)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 18, 19, 20, 20, 20, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 33, 34, 35, 36, 37, 37, 38, 39, 40, 41, 42, 43, 44, 44, 44, 45, 46, 47, 48, 49, 49, 49, 49, 50, 51, 52, 53, 54, 55, 56, 56, 56, 57, 58, 58, 58, 59, 60, 61, 62, 63, 64, 65, 66, 66, 67, 68, 69, 69, 70, 71, 72, 73, 74, 75, 76, 77, 77, 78, 79, 80, 81, 82, 83, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 113, 114, 115, 116, 117, 118, 118]


In [ ]:
np.argmax(start_logits_context)

26

In [ ]:
start_word_id = context_tok_to_word_id[np.argmax(start_logits_context)]
start_word_id

22

In [ ]:
end_word_id = context_tok_to_word_id[np.argmax(end_logits_context)]
end_word_id

37

RESPOSTA FINAL

In [ ]:
predicted_answer = " ".join(my_context_words[start_word_id:end_word_id + 1])
print("The answer to:\n" + my_question + "\nis:\n" + predicted_answer)

The answer to:
What is civil disobedience?
is:
one of the many ways people have rebelled against what they deem to be unfair laws.


In [ ]:
from IPython.core.display import HTML
display(HTML(f'<h2>{my_question.upper()}</h2>'))
marked_text = str(my_context.replace(predicted_answer, f"<mark>{predicted_answer}</mark>"))
#print(marked_text)
display(HTML(f"""<blockquote> {marked_text} </blockquote>"""))